In [1]:
! pwd

/home/guillermo.carrilho/PhysicsSimulationDeepLearning


In [1]:
import os
import sys


ROOT="/home/guillermo.carrilho/PhysicsSimulationDeepLearning"

sys.path.append(os.path.join(ROOT,"Physical_models"))

In [2]:
from Differentiable_simulation import two_phase_flow,dK_w,K_w,K_o
from phi.torch.flow import *


geo_w=UniformGrid(x=32, y=32)
phi_w=Field(geo_w,values=tensor(0.1),
      boundary= {
          'x-':0.5,
          'x+': ZERO_GRADIENT,
          'y-': ZERO_GRADIENT,
          'y+': ZERO_GRADIENT
 })

geo_o=UniformGrid(x=32, y=32)
phi_o=Field(geo_o,values=tensor(0.0),
      boundary= {
          'x-': ZERO_GRADIENT,
          'x+': 0.5,
          'y-': ZERO_GRADIENT,
          'y+': ZERO_GRADIENT
 })

phy=two_phase_flow(
    phi_w,
    phi_o,
    dt=0.01,
    advection_solver=lambda v: Solve('CG-adaptive',1e-3,1e-3,x0=v)
)

In [3]:
phy.compute_p_c(phy.phi_o,phy.phi_o)
phy.momentum_eq(u=phy.phi_o,dt=0.1)

/usr/local/lib/python3.10/dist-packages/phiml/backend/torch/_torch_backend.py:837: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(row_pointers, column_indices, values, shape, device=values.device)


Field[(xˢ=32, yˢ=32, vectorᶜ=x,y)]

In [4]:
advect.semi_lagrangian(field.spatial_gradient(phi_o),
                       phy.compute_convective_velocity(phy.phi_w,phy.phi_o,dK_w),0.1)

#field.spatial_gradient(phy.phi_o)

Field[(xˢ=32, yˢ=32, vectorᶜ=x,y)]

In [18]:
#from Differentiable_simulation import K_w,K_o
import anisotropic_diffusion

#differential(u=phy.phi_o,diffusivity=K_o(phy.compute_p_c(phy.phi_w,phy.phi_o)))

#explicit(u=phy.phi_o,diffusivity=K_o(phy.compute_p_c(phy.phi_w,phy.phi_o)),dt=0.1)


anisotropic_diffusion.implicit(phy.phi_o,
             diffusivity=K_o(phy.compute_p_c(phy.phi_w,phy.phi_o))*1e-8,
             dt=0.001,solve=Solve('CG'))


Field[(xˢ=32, yˢ=32)]

In [22]:
#v = math.solve_linear(self.momentum_eq, v, self.advection_solver(v), dt=-dt)
stack([phy.phi_o,phy.phi_w],"a")

/usr/local/lib/python3.10/dist-packages/phiml/math/_magic_ops.py:259: UserWarning: Failed to concat values using value attributes because attributes differ among values [float64 0.5, zero-gradient]
  warnings.warn(f"Failed to concat values using value attributes because attributes differ among values {values}")


MagicNotImplemented: At least one item in values must be Shapable but got types [<class 'phiml.math.extrapolation.ConstantExtrapolation'>, <class 'phiml.math.extrapolation._ZeroGradient'>]

In [98]:
geo_w=UniformGrid(x=32, y=32)
#phi_comp=Field(geo_w,values=stack([vec("w",x=0.1,y=0.1),vec("w",x=0.1,y=0.1)],dim="p"),
#phi_comp=(geo_w,values=Field(geo_w,values=vec(w=vec("w",x=0.1,y=0.1),o=vec("o",x=0.1,y=0.1)),dim="p"),
phi_comp=Field(geo_w,values=vec(w=vec("w",x=0.1,y=0.1),o=vec("o",x=0.1,y=0.1)),dim="p",
boundary= {"w":{
          'x-':ZERO_GRADIENT,
          'x+': vec(x=0.1,y=0.1),
          'y-': ZERO_GRADIENT,
          'y+': ZERO_GRADIENT},
          "o":{
          'x-':vec(x=0.1,y=0.1),
          'x+': ZERO_GRADIENT,
          'y-': ZERO_GRADIENT,
          'y+': ZERO_GRADIENT}
          }
          
          )
phi_comp

Field[(wˢ=x,y, oˢ=x,y, vectorᶜ=w,o, xˢ=32, yˢ=32)]

In [103]:
phi_.extrapolation.determines_boundary_values("x-")

#math.extrapolation.determines_boundary_values(phi_comp)

NameError: name 'phi_' is not defined

In [72]:

#stack([vec("w",x=0.1,y=0.1),vec("o",x=0.1,y=0.1)],"w,o")
vec(w=vec("w",x=0.1,y=0.1),o=vec("o",x=0.1,y=0.1))
#tensor?

Dense.wˢ=x,y

In [69]:
vec(x=0.1,y=0.1)

(x=0.100, y=0.100) float64

In [46]:
phi_comp.boundary

{'x-': (x=0.100, y=0.100); (x=0.100, y=0.100) (pˢ=2, vectorᶜ=x,y), 'x+': zero-gradient, 'y-': zero-gradient, 'y+': zero-gradient}

In [83]:
from phiml import math
math.extrapolation.as_extrapolation(
    {"w":{"y":0,"x":0},
     "o":{"y":0,"x":0.1}
     }
    )

AssertionError: Nested mixed extrapolations not supported

# Gradient Notes

In [ ]:
import torch

### Grandient example

In [ ]:
x1 = torch.ones((3,1,3)).requires_grad_(True)  #
print(x1)

u1 = torch.stack([
    x1[:,:,0]**0.5,
    torch.sin(x1[:,:,1])+x1[:,:,0]**0.5,
    torch.cos(x1[:,:,0]),
    x1[:,:,1]**2,
    ],axis=2)

#u1 = torch.matmul(M,x1)


#u1=torch.sin(torch.matmul(torch.rand(2,2).requires_grad_(True),x1))
print(u1)
print(u1.shape)
torch.autograd.grad(
    u1,x1,
            grad_outputs=torch.ones_like(u1).to(u1.device),
            create_graph=True,
            retain_graph=True,
            allow_unused=True
            )

tensor([[[1., 1., 1.]],

        [[1., 1., 1.]],

        [[1., 1., 1.]]], requires_grad=True)
tensor([[[1.0000, 1.8415, 0.5403, 1.0000]],

        [[1.0000, 1.8415, 0.5403, 1.0000]],

        [[1.0000, 1.8415, 0.5403, 1.0000]]], grad_fn=<StackBackward0>)
torch.Size([3, 1, 4])


(tensor([[[0.1585, 2.5403, 0.0000]],
 
         [[0.1585, 2.5403, 0.0000]],
 
         [[0.1585, 2.5403, 0.0000]]], grad_fn=<AddBackward0>),)

### Jacobian computation

Expressed in pytorch with torch.autograd.grad

$$ \textbf{u} \in \mathbb{R}^n \\ \textbf{x},\textbf{e}_i \in \mathbb{R}^m $$
$$ T_{ag} (\textbf{u},\textbf{x},\textbf{e}_i) = \nabla_{\textbf{x}} \textbf{u}_i  $$
$$ T_{ag} (\textbf{u},\textbf{x},\textbf{e}_i) = \nabla_{\textbf{x}} \textbf{u}_i  $$
$$ T_{ag} (\textbf{u},\textbf{x},\textbf{e}_i+\textbf{e}_j) = \nabla_{\textbf{x}} \textbf{u}_i + \nabla_{\textbf{x}} \textbf{u}_j $$
$$ [T_{ag} (\textbf{u},\textbf{x},\textbf{e}_i)]_{i=1}^n = J_{\textbf{x}}(\textbf{u})$$


In [ ]:
unit_vectors=torch.eye(4)
print(u1)
jacobian_rows = [torch.autograd.grad(u1, x1, vec_.unsqueeze(0).unsqueeze(0).tile(u1.shape[0],u1.shape[1],1),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
                     for vec_ in unit_vectors]
print(jacobian_rows)
print(torch.stack(jacobian_rows,axis=2))
print(torch.stack(jacobian_rows,axis=2).shape)

torch.autograd.grad(u1, x1, unit_vectors[1].unsqueeze(0).unsqueeze(0).tile(u1.shape[0],u1.shape[1],1),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0].shape

tensor([[[1.0000, 1.8415, 0.5403, 1.0000]],

        [[1.0000, 1.8415, 0.5403, 1.0000]],

        [[1.0000, 1.8415, 0.5403, 1.0000]]], grad_fn=<StackBackward0>)
[tensor([[[0.5000, 0.0000, 0.0000]],

        [[0.5000, 0.0000, 0.0000]],

        [[0.5000, 0.0000, 0.0000]]], grad_fn=<AddBackward0>), tensor([[[0.5000, 0.5403, 0.0000]],

        [[0.5000, 0.5403, 0.0000]],

        [[0.5000, 0.5403, 0.0000]]], grad_fn=<AddBackward0>), tensor([[[-0.8415,  0.0000,  0.0000]],

        [[-0.8415,  0.0000,  0.0000]],

        [[-0.8415,  0.0000,  0.0000]]], grad_fn=<AddBackward0>), tensor([[[0., 2., 0.]],

        [[0., 2., 0.]],

        [[0., 2., 0.]]], grad_fn=<AddBackward0>)]
tensor([[[[ 0.5000,  0.0000,  0.0000],
          [ 0.5000,  0.5403,  0.0000],
          [-0.8415,  0.0000,  0.0000],
          [ 0.0000,  2.0000,  0.0000]]],


        [[[ 0.5000,  0.0000,  0.0000],
          [ 0.5000,  0.5403,  0.0000],
          [-0.8415,  0.0000,  0.0000],
          [ 0.0000,  2.0000,  0.0000]]],


 

torch.Size([3, 1, 3])

In [ ]:
#

def vector_jacobian(u,x):
    unit_vectors=torch.eye(u.shape[-1])
    jacobian_rows = [torch.autograd.grad(u, x, vec_.unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
                     for vec_ in unit_vectors]
    return torch.stack(jacobian_rows,axis=2)


def vector_grad(u,x):
    unit_vectors=torch.eye(u.shape[-1])
    jacobian_rows = [torch.autograd.grad(u, x, vec_.unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
                     for vec_ in unit_vectors]
    return torch.diagonal(torch.stack(jacobian_rows,axis=2),dim1=-2,dim2=-1)

def x_grad(u,x,i,n):
    """
    gradient of degree wrt x for componen i for u
    input:
    u and x are tensors with vectors object at dimension -1
    [b, n_vectors, vector_dimension]

    output:
    [b, n_vectors, input_vector_dimension]
    """
    I=torch.eye(u.shape[-1])

    u=torch.autograd.grad(u ,x,
            I[i].unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1).to(u.device),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
    if n > 1:
        for i in range(n-1):
            u=vector_grad(u,x)
    return u

In [ ]:
print(x_grad(u1,x1,0,1))
udx=x_grad(u1,x1,0,1)


tensor([[[0.5000, 0.0000, 0.0000]],

        [[0.5000, 0.0000, 0.0000]],

        [[0.5000, 0.0000, 0.0000]]], grad_fn=<AddBackward0>)


In [ ]:
print(x1.shape)
x1

torch.Size([3, 1, 3])


tensor([[[1., 1., 1.]],

        [[1., 1., 1.]],

        [[1., 1., 1.]]], requires_grad=True)

In [ ]:
torch.sum(x1*udx,axis=-1)

tensor([[0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SumBackward1>)

In [ ]:
udx[:,:,:2]

tensor([[[0.5000, 0.0000]],

        [[0.5000, 0.0000]],

        [[0.5000, 0.0000]]], grad_fn=<SliceBackward0>)

# Residuals debug

## NS reesidual

In [ ]:
def incompresibble_fluid_loss(up,xt,mu=1,rho=1):
    l=0
    # x-velocity components
    l+=x_grad(up,xt,0,1)[...,2] # dudt
    l+=torch.sum(up[...,:1]*x_grad(up,xt,0,1)[...,:2],axis=-1) # u * grad u
    l+=(mu/rho)*(x_grad(up,xt,2,1)[...,0]) #  dpdx
    l-=(mu/rho)*torch.sum(x_grad(up,xt,0,2)[...,:2],axis=-1) # grad**2 u
    # y-velocity components
    l+=x_grad(up,xt,1,1)[...,2] # dvdt
    l+=torch.sum(up[...,1:2]*x_grad(up,xt,0,1)[...,:2],axis=-1) # v * grad v
    l+=(mu/rho)*(x_grad(up,xt,2,1)[...,1]) #  dpdy
    l-=(mu/rho)*torch.sum(x_grad(up,xt,1,2)[...,:2],axis=-1) # grad**2 v
    return l

### Debug

In [ ]:
x1 = torch.randn((3,1,3)).requires_grad_(True)  #
print(x1)
print(x1.shape)

u1 = torch.stack([
    x1[:,:,0]**0.5,
    torch.sin(x1[:,:,1])+x1[:,:,0]**0.5,
    torch.cos(x1[:,:,0]),
    ],axis=2)

print(u1.shape)

tensor([[[-1.5316, -0.0218,  1.3527]],

        [[ 2.0246,  0.5625, -0.8633]],

        [[ 0.5488, -0.9313,  0.3730]]], requires_grad=True)
torch.Size([3, 1, 3])
torch.Size([3, 1, 3])


In [ ]:
incompresibble_fluid_loss(u1,x1,1,1)-0
#x_grad(u1,x1,0,1)[...,0]
#print(x1[:,:,:1])
#print(x_grad(u1,x1,0,1)[...,:2])
#x_grad(u1,x1,0,1)[...,2]
#x_grad(u1,x1,0,2)[...,:2]

tensor([[   nan],
        [0.9955],
        [0.3642]], grad_fn=<SubBackward0>)